In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
import os

In [3]:
fitles = pd.read_csv('../data/fourgram_titles_pos_w2v.csv')
fitles.head()

,title,good_title,pos,pos1,pos2,pos3,pos4,title_token,word_vec
0,with him ? knows,0,"['IN', 'PRP', '.', 'NNS']",IN,PRP,.,NNS,"['with', 'him', '?', 'knows']",[[-7.74908543e-01 7.73788244e-02 5.28766096e...
1,is good for your,0,"['VBZ', 'JJ', 'IN', 'PRP$']",VBZ,JJ,IN,PRP$,"['is', 'good', 'for', 'your']",[[-8.93845335e-02 -5.09242773e-01 -1.35309592e...
2,man ? thy form,0,"['NN', '.', 'JJ', 'NN']",NN,.,JJ,NN,"['man', '?', 'thy', 'form']",[[-1.02470890e-01 -1.80637687e-01 -1.06548798e...
3,traitorous father are not,0,"['JJ', 'NN', 'VBP', 'RB']",JJ,NN,VBP,RB,"['traitorous', 'father', 'are', 'not']",[[ 1.47299841e-02 -5.07809073e-02 3.13400924e...
4,"daintiest last , to",0,"['JJS', 'JJ', ',', 'TO']",JJS,JJ,",",TO,"['daintiest', 'last', ',', 'to']",[[ 2.92296102e-03 -7.05268141e-03 -5.92417922e...


In [4]:
fourshake = pd.read_csv('../data/shakespearean_fourgrams_pos.csv')
fourshake.sample(5)

,title,title_tokenized,pos_tag,pos,pos_len,pos1,pos2,pos3,pos4
64394,"bestow her funeral ,","['bestow', 'her', 'funeral', ',']","['IN', 'PRP$', 'JJ', ',']","['IN', 'PRP$', 'JJ', ',']",4,IN,PRP$,JJ,","
318084,"others touch , and","['others', 'touch', ',', 'and']","['NNS', 'JJ', ',', 'CC']","['NNS', 'JJ', ',', 'CC']",4,NNS,JJ,",",CC
89227,come in my closet,"['come', 'in', 'my', 'closet']","['VBN', 'IN', 'PRP$', 'NN']","['VBN', 'IN', 'PRP$', 'NN']",4,VBN,IN,PRP$,NN
69856,break into a hundred,"['break', 'into', 'a', 'hundred']","['NN', 'IN', 'DT', 'JJ']","['NN', 'IN', 'DT', 'JJ']",4,NN,IN,DT,JJ
69366,"boys , are we","['boys', ',', 'are', 'we']","['NNS', ',', 'VBP', 'PRP']","['NNS', ',', 'VBP', 'PRP']",4,NNS,",",VBP,PRP


The fourgrams still need word embedding

In [5]:
fourshake = fourshake.drop(columns=['title_tokenized', 'pos_tag', 'pos_len'])

In [6]:
twd = TreebankWordTokenizer()

fourshake['title_token'] = fourshake['title'].apply(lambda x: twd.tokenize(x))
fourshake.sample(5)

,title,pos,pos1,pos2,pos3,pos4,title_token
357540,shillings i won of,"['NNS', 'VBP', 'NNS', 'IN']",NNS,VBP,NNS,IN,"[shillings, i, won, of]"
473665,"we have digested ,","['PRP', 'VBP', 'VBN', ',']",PRP,VBP,VBN,",","[we, have, digested, ,]"
418228,then he is alive,"['RB', 'PRP', 'VBZ', 'JJ']",RB,PRP,VBZ,JJ,"[then, he, is, alive]"
125788,feel only looking on,"['NN', 'RB', 'VBG', 'IN']",NN,RB,VBG,IN,"[feel, only, looking, on]"
279411,my sorrows to be,"['PRP$', 'NNS', 'TO', 'VB']",PRP$,NNS,TO,VB,"[my, sorrows, to, be]"


Now to replace every word in each trigram with the learned word vector

In [7]:
wv = KeyedVectors.load("../feature_engineering/shake_w2v.wordsvectors", mmap='r')

In [8]:
words = set(wv.index2word)
len(words)

26304

In [11]:
fourgram_vect = [np.array([wv[i] for i in ls if i in words])
                                for ls in fourshake['title_token']]

In [15]:
fourshake['word_vec'] = fourgram_vect

In [18]:
loss_title_token = []

for i,v in enumerate(fourgram_vect):
    if len(v) != 4:
        loss_title_token.append(i)

print('Number of fourgrams with no word matches from title_token', len(loss_title_token))

Number of fourgrams with no word matches from title_token 17181


Previous wrod has shown that many of these title losses are due to pwrds with a period at the end, such as:
`a bawd. hold` or ` about him. in`. These occur at sentence breaks, which are areas the we would not want titles generated from.
    

In [20]:
fourshake.iloc[4626]

title                                        a full soldier. let
pos                                     ['DT', 'JJ', 'NN', 'NN']
pos1                                                          DT
pos2                                                          JJ
pos3                                                          NN
pos4                                                          NN
title_token                             [a, full, soldier., let]
word_vec       [[0.0037827007, 0.38528407, -0.89005077, 0.681...
Name: 4626, dtype: object

In [21]:
fourshake.drop(index=loss_title_token, inplace=True)
fourshake.shape

(511221, 8)

In [22]:
fourshake.reset_index(drop=True, inplace=True)
fourshake.sample(3)

,title,pos,pos1,pos2,pos3,pos4,title_token,word_vec
156790,have ta'en you at,"['VB', 'NN', 'PRP', 'IN']",VB,NN,PRP,IN,"[have, ta'en, you, at]","[[0.71699846, -0.31489325, -0.50631213, -1.697..."
103436,do not see them,"['VB', 'RB', 'VB', 'PRP']",VB,RB,VB,PRP,"[do, not, see, them]","[[-0.5646246, -0.51202106, -1.1728927, 0.00235..."
19752,"amendment , lords ?","['NN', ',', 'NNS', '.']",NN,",",NNS,.,"[amendment, ,, lords, ?]","[[0.0029164026, -0.009978041, -0.0039208992, -..."


I need to to be very careful with the one hot encoding because the title and shakespeare datasets will have different pos tags. I'll need to filter out all the pos tags in the shakespeare dataset that do not appear in the titles dataset, and visa versa

In [23]:
all_fitles_pos1 = set(fitles.pos1.tolist())
all_fitles_pos2 = set(fitles.pos2.tolist())
all_fitles_pos3 = set(fitles.pos3.tolist())
all_fitles_pos4 = set(fitles.pos4.tolist())


print('Number of pos tags in titles pos1:', len(all_fitles_pos1))
print('Number of pos tags in titles pos2:', len(all_fitles_pos2))
print('Number of pos tags in titles pos3:', len(all_fitles_pos3))
print('Number of pos tags in titles pos4:', len(all_fitles_pos4))

all_fourshake_pos1 = set(fourshake.pos1.tolist())
all_fourshake_pos2 = set(fourshake.pos2.tolist())
all_fourshake_pos3 = set(fourshake.pos3.tolist())
all_fourshake_pos4 = set(fourshake.pos4.tolist())

print('Number of pos tags in shakespeare pos1:', len(all_fourshake_pos1))
print('Number of pos tags in shakespeare pos2:', len(all_fourshake_pos2))
print('Number of pos tags in shakespeare pos3:', len(all_fourshake_pos3))
print('Number of pos tags in shakespeare pos4:', len(all_fourshake_pos4))

Number of pos tags in titles pos1: 23
Number of pos tags in titles pos2: 27
Number of pos tags in titles pos3: 24
Number of pos tags in titles pos4: 24
Number of pos tags in shakespeare pos1: 32
Number of pos tags in shakespeare pos2: 38
Number of pos tags in shakespeare pos3: 38
Number of pos tags in shakespeare pos4: 37


In [24]:
pos1_rm = list(all_fourshake_pos1.difference(all_fitles_pos1))
print(pos1_rm)

pos2_rm = list(all_fourshake_pos2.difference(all_fitles_pos2))
print(pos2_rm)

pos3_rm = list(all_fourshake_pos3.difference(all_fitles_pos3))
print(pos3_rm)

pos4_rm = list(all_fourshake_pos4.difference(all_fitles_pos4))
print(pos4_rm)

pos5_rm = list(all_fitles_pos1.difference(all_fourshake_pos1))
print(pos5_rm)

pos6_rm = list(all_fitles_pos2.difference(all_fourshake_pos2))
print(pos6_rm)

pos7_rm = list(all_fitles_pos3.difference(all_fourshake_pos3))
print(pos7_rm)

pos8_rm = list(all_fitles_pos4.difference(all_fourshake_pos4))
print(pos8_rm)

['UH', 'FW', 'JJR', 'PDT', 'RBS', 'VBP', 'WP$', '.', 'RP']
['CD', 'WP', 'NNP', 'UH', 'WDT', 'FW', 'JJR', 'NNPS', 'WP$', ':', '""']
['RBR', 'EX', 'NNP', 'UH', 'WDT', 'JJR', 'PDT', 'NNPS', 'VBG', 'JJS', 'RBS', 'WP$', ':', '""']
['CD', 'RBR', 'WP', 'UH', 'WDT', 'FW', 'PDT', 'JJS', 'SYM', 'RBS', 'WP$', 'VBG', '""']
[]
[]
[]
[]


In [25]:
print('Starting number of fourgrams', fourshake.shape[0])
fourshake = fourshake[~fourshake['pos1'].isin(pos1_rm)]
fourshake.shape[0]

Starting number of fourgrams 511221


501598

In [26]:
print('Starting number of fourgrams', fourshake.shape[0])
fourshake = fourshake[~fourshake['pos2'].isin(pos2_rm)]
fourshake.shape[0]

Starting number of fourgrams 501598


492053

In [27]:
print('Starting number of fourgrams', fourshake.shape[0])
fourshake = fourshake[~fourshake['pos3'].isin(pos3_rm)]
fourshake.shape[0]

Starting number of fourgrams 492053


479694

In [28]:
print('Starting number of fourgrams', fourshake.shape[0])
fourshake = fourshake[~fourshake['pos4'].isin(pos4_rm)]
fourshake.shape[0]

Starting number of fourgrams 479694


468606

In [29]:
all_fitles_pos1 = set(fitles.pos1.tolist())
all_fitles_pos2 = set(fitles.pos2.tolist())
all_fitles_pos3 = set(fitles.pos3.tolist())
all_fitles_pos4 = set(fitles.pos4.tolist())


print('Number of pos tags in titles pos1:', len(all_fitles_pos1))
print('Number of pos tags in titles pos2:', len(all_fitles_pos2))
print('Number of pos tags in titles pos3:', len(all_fitles_pos3))
print('Number of pos tags in titles pos4:', len(all_fitles_pos4))

all_fourshake_pos1 = set(fourshake.pos1.tolist())
all_fourshake_pos2 = set(fourshake.pos2.tolist())
all_fourshake_pos3 = set(fourshake.pos3.tolist())
all_fourshake_pos4 = set(fourshake.pos4.tolist())

print('Number of pos tags in shakespeare pos1:', len(all_fourshake_pos1))
print('Number of pos tags in shakespeare pos2:', len(all_fourshake_pos2))
print('Number of pos tags in shakespeare pos3:', len(all_fourshake_pos3))
print('Number of pos tags in shakespeare pos4:', len(all_fourshake_pos4))

Number of pos tags in titles pos1: 23
Number of pos tags in titles pos2: 27
Number of pos tags in titles pos3: 24
Number of pos tags in titles pos4: 24
Number of pos tags in shakespeare pos1: 23
Number of pos tags in shakespeare pos2: 27
Number of pos tags in shakespeare pos3: 24
Number of pos tags in shakespeare pos4: 24


In [30]:
pos1_rm = list(all_fourshake_pos1.difference(all_fitles_pos1))
print(pos1_rm)

pos2_rm = list(all_fourshake_pos2.difference(all_fitles_pos2))
print(pos2_rm)

pos3_rm = list(all_fourshake_pos3.difference(all_fitles_pos3))
print(pos3_rm)

pos4_rm = list(all_fourshake_pos4.difference(all_fitles_pos4))
print(pos4_rm)

[]
[]
[]
[]


POS tags synced! Now to create dummy variables from the POS tags.

In [31]:
fitles_pos = fitles[['pos1', 'pos2', 'pos3', 'pos4']]
fitles_pos = pd.get_dummies(fitles_pos, prefix=['pos1', 'pos2', 'pos3', 'pos4'], columns=['pos1', 'pos2', 'pos3', 'pos4'])
print('fitles_pos.shape:', fitles_pos.shape)
fitles_pos.sample()

fitles_pos.shape: (338, 98)


,pos1_CC,pos1_CD,pos1_DT,pos1_EX,pos1_IN,pos1_JJ,pos1_JJS,pos1_MD,pos1_NN,pos1_NNS,...,pos4_PRP$,pos4_RB,pos4_RP,pos4_TO,pos4_VB,pos4_VBD,pos4_VBN,pos4_VBP,pos4_VBZ,pos4_WRB
276,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
all_fitles_pos = fitles_pos.values.tolist()
print('all_fitles_pos list of lists length:', len(all_fitles_pos))
print('length of inner lists aka number of pos tags:', len(all_fitles_pos[0]))

all_fitles_pos list of lists length: 338
length of inner lists aka number of pos tags: 98


In [33]:
fourshake_pos = fourshake[['pos1', 'pos2', 'pos3', 'pos4']]
fourshake_pos = pd.get_dummies(fourshake_pos, prefix=['pos1', 'pos2', 'pos3', 'pos4'], columns=['pos1', 'pos2', 'pos3', 'pos4'])
print('fourshake_pos.shape:', fourshake_pos.shape)
fourshake_pos.sample()

fourshake_pos.shape: (468606, 98)


,pos1_CC,pos1_CD,pos1_DT,pos1_EX,pos1_IN,pos1_JJ,pos1_JJS,pos1_MD,pos1_NN,pos1_NNS,...,pos4_PRP$,pos4_RB,pos4_RP,pos4_TO,pos4_VB,pos4_VBD,pos4_VBN,pos4_VBP,pos4_VBZ,pos4_WRB
8809,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
all_fourshake_pos = fourshake_pos.values.tolist()
print('all_fourshake_pos list of lists length:', len(all_fourshake_pos))
print('length of inner lists aka number of pos tags:', len(all_fourshake_pos[0]))

all_fourshake_pos list of lists length: 468606
length of inner lists aka number of pos tags: 98


In [35]:
title_col = fitles_pos.columns
shake_col = fourshake_pos.columns

for i in range(0, 98):
    if title_col[i] != shake_col[i]:
        print('Alas!', title_col[i], shake_col[i])

In [36]:
fitles['all_pos'] = all_fitles_pos
fitles.sample()

,title,good_title,pos,pos1,pos2,pos3,pos4,title_token,word_vec,all_pos
314,speak daggers to her,1,"['NN', 'NNS', 'TO', 'PRP$']",NN,NNS,TO,PRP$,"['speak', 'daggers', 'to', 'her']",[[-2.09728375e-01 -3.97513092e-01 -1.20670378e...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [37]:
fourshake['all_pos'] = all_fourshake_pos
fourshake.sample()

,title,pos,pos1,pos2,pos3,pos4,title_token,word_vec,all_pos
320167,present medicine must be,"['JJ', 'NN', 'MD', 'VB']",JJ,NN,MD,VB,"[present, medicine, must, be]","[[-0.32386544, -0.04748271, -0.08426321, -0.33...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


And now to create a second dataset by removing the first one-hot encoded category

In [38]:
fitles_pos2 = fitles[['pos1', 'pos2', 'pos3', 'pos4']]
fitles_pos2 = pd.get_dummies(fitles_pos2, prefix=['pos1', 'pos2', 'pos3', 'pos4'], columns=['pos1', 'pos2', 'pos3', 'pos4'], drop_first=True)
print('fitles_pos2.shape:', fitles_pos2.shape)
fitles_pos2.sample()

fitles_pos2.shape: (338, 94)


,pos1_CD,pos1_DT,pos1_EX,pos1_IN,pos1_JJ,pos1_JJS,pos1_MD,pos1_NN,pos1_NNS,pos1_PRP,...,pos4_PRP$,pos4_RB,pos4_RP,pos4_TO,pos4_VB,pos4_VBD,pos4_VBN,pos4_VBP,pos4_VBZ,pos4_WRB
131,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [39]:
dp1_fitles_pos = fitles_pos2.values.tolist()
print('dp1_fitles_pos list of lists length:', len(dp1_fitles_pos))
print('length of inner lists aka number of pos tags:', len(dp1_fitles_pos[0]))

dp1_fitles_pos list of lists length: 338
length of inner lists aka number of pos tags: 94


In [40]:
fourshake_pos2 = fourshake[['pos1', 'pos2', 'pos3', 'pos4']]
fourshake_pos2 = pd.get_dummies(fourshake_pos2, prefix=['pos1', 'pos2', 'pos3', 'pos4'], columns=['pos1', 'pos2', 'pos3', 'pos4'], drop_first=True)
print('fourshake_pos2.shape:', fourshake_pos2.shape)
fourshake_pos2.sample()

fourshake_pos2.shape: (468606, 94)


,pos1_CD,pos1_DT,pos1_EX,pos1_IN,pos1_JJ,pos1_JJS,pos1_MD,pos1_NN,pos1_NNS,pos1_PRP,...,pos4_PRP$,pos4_RB,pos4_RP,pos4_TO,pos4_VB,pos4_VBD,pos4_VBN,pos4_VBP,pos4_VBZ,pos4_WRB
484945,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
dp1_fourshake_pos = fourshake_pos2.values.tolist()
print('dp1_fourshake_pos list of lists length:', len(dp1_fourshake_pos))
print('length of inner lists aka number of pos tags:', len(dp1_fourshake_pos[0]))

dp1_fourshake_pos list of lists length: 468606
length of inner lists aka number of pos tags: 94


In [42]:
title_col2 = fitles_pos2.columns
shake_col2 = fourshake_pos2.columns

for i in range(0, 78):
    if title_col2[i] != shake_col2[i]:
        print('Alas!', title_col2[i], shake_col2[i])

In [43]:
fitles['dp1_pos'] = dp1_fitles_pos
fitles.sample()

,title,good_title,pos,pos1,pos2,pos3,pos4,title_token,word_vec,all_pos,dp1_pos
110,"thing , yea ,",0,"['NN', ',', 'NN', ',']",NN,",",NN,",","['thing', ',', 'yea', ',']",[[-1.03537709e-01 -3.90889376e-01 -5.70682108e...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [44]:
fourshake['dp1_pos'] = dp1_fourshake_pos
fourshake.sample()

,title,pos,pos1,pos2,pos3,pos4,title_token,word_vec,all_pos,dp1_pos
104299,do you mightily hold,"['VB', 'PRP', 'RB', 'VB']",VB,PRP,RB,VB,"[do, you, mightily, hold]","[[-0.5646246, -0.51202106, -1.1728927, 0.00235...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."


In [45]:
datapath = '../data'

datapath_fourshake = os.path.join(datapath, 'shakespearean_fourgrams.pkl')
fourshake.to_pickle(datapath_fourshake)

datapath_fitles = os.path.join(datapath, 'fourgram_titles.pkl')
fitles.to_pickle(datapath_fitles)

In [1]:
import pandas as pd
import numpy as np
import os

In [4]:
fitles = pd.read_pickle('../data/fourgram_titles.pkl')
fitles.head()

,title,good_title,pos,pos1,pos2,pos3,pos4,title_token,word_vec,all_pos,dp1_pos
0,with him ? knows,0,"['IN', 'PRP', '.', 'NNS']",IN,PRP,.,NNS,"['with', 'him', '?', 'knows']",[[-7.74908543e-01 7.73788244e-02 5.28766096e...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,is good for your,0,"['VBZ', 'JJ', 'IN', 'PRP$']",VBZ,JJ,IN,PRP$,"['is', 'good', 'for', 'your']",[[-8.93845335e-02 -5.09242773e-01 -1.35309592e...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,man ? thy form,0,"['NN', '.', 'JJ', 'NN']",NN,.,JJ,NN,"['man', '?', 'thy', 'form']",[[-1.02470890e-01 -1.80637687e-01 -1.06548798e...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,traitorous father are not,0,"['JJ', 'NN', 'VBP', 'RB']",JJ,NN,VBP,RB,"['traitorous', 'father', 'are', 'not']",[[ 1.47299841e-02 -5.07809073e-02 3.13400924e...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"daintiest last , to",0,"['JJS', 'JJ', ',', 'TO']",JJS,JJ,",",TO,"['daintiest', 'last', ',', 'to']",[[ 2.92296102e-03 -7.05268141e-03 -5.92417922e...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
fitles = fitles.drop(columns=['pos', 'pos1', 'pos2', 'pos3', 'pos4', 'title_token'])
fitles.sample()

,title,good_title,word_vec,all_pos,dp1_pos
333,the eye of heaven,1,[[-9.59224224e-01 1.22396159e+00 -6.39906406e...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
type(fitles.iloc[0,2])

str

Now to reformat the `word_vec` column from a string to an 800 dimension word vector.

In [9]:
result1 = fitles['word_vec'].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))
type(result1[0])

numpy.ndarray

In [11]:
len(result1[1])

800

In [15]:
fitles['result'] = result1
fitles.sample()

,title,good_title,word_vec,all_pos,dp1_pos,result
308,trippingly on the tongue,1,[[-9.52565484e-03 -1.08746914e-02 -1.34599209e...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[-0.00952565484, -0.0108746914, -0.0134599209,..."


In [17]:
fitles['w2v'] = fitles['result'].apply(lambda x: list(x))
fitles.sample()

,title,good_title,word_vec,all_pos,dp1_pos,result,w2v
171,no enemy but winter,1,[[-8.46432894e-02 8.84881377e-01 -4.20693010e...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.0846432894, 0.884881377, -0.42069301, -0.1...","[-0.0846432894, 0.884881377, -0.42069301, -0.1..."


In [21]:
fitles = fitles.drop(columns=['word_vec', 'result'])

In [20]:
fitles.sample()

,title,good_title,word_vec,all_pos,dp1_pos,result,w2v
77,any good thing to,0,[[-2.20608994e-01 2.12935314e-01 -3.57046336e...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.220608994, 0.212935314, -0.357046336, -0.1...","[-0.220608994, 0.212935314, -0.357046336, -0.1..."


Now combine `w2v` with `all_pos` and `dp1_pos`

In [22]:
w2v_dp1_pos = fitles[['w2v', 'dp1_pos']]
w2v_dp1 = w2v_dp1_pos.values.tolist()

print('Length of w2v_dp1 list:', len(w2v_dp1))
print('Length of first element of w2v_dp1 list:', len(w2v_dp1[0]))
print('Length of first element of inner list:', len(w2v_dp1[0][0]))
print('Type of first element of inner list:', type(w2v_dp1[0][0]))
print('Length of seceond element of inner list', len(w2v_dp1[0][1]))
print('Type of element of inner list', type(w2v_dp1[0][1]))

Length of w2v_dp1 list: 338
Length of first element of w2v_dp1 list: 2
Length of first element of inner list: 800
Type of first element of inner list: <class 'list'>
Length of seceond element of inner list 94
Type of element of inner list <class 'list'>


In [23]:
new_w2v_dp1 = []

for i in range(len(w2v_dp1)):
    new_w2v_dp1.append(w2v_dp1[i][0] + w2v_dp1[i][1])
    
print('Length of new_w2v_dp1', len(new_w2v_dp1))
print('Length of inner list', len(new_w2v_dp1[0]))

Length of new_w2v_dp1 338
Length of inner list 894


In [24]:
fitles['w2v_dp1'] = new_w2v_dp1
fitles.sample()

,title,good_title,all_pos,dp1_pos,w2v,w2v_dp1
221,"out, brief candle",1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.939932466, 0.547169149, -0.860268235, 0.06...","[-0.939932466, 0.547169149, -0.860268235, 0.06..."


In [25]:
w2v_all_pos = fitles[['w2v', 'all_pos']]
w2v_all = w2v_all_pos.values.tolist()

print('Length of w2v_all list:', len(w2v_dp1))
print('Length of first element of w2v_all list:', len(w2v_all[0]))
print('Length of first element of inner list:', len(w2v_all[0][0]))
print('Type of first element of inner list:', type(w2v_all[0][0]))
print('Length of seceond element of inner list', len(w2v_all[0][1]))
print('Type of element of inner list', type(w2v_all[0][1]))

Length of w2v_all list: 338
Length of first element of w2v_all list: 2
Length of first element of inner list: 800
Type of first element of inner list: <class 'list'>
Length of seceond element of inner list 98
Type of element of inner list <class 'list'>


In [26]:
new_w2v_all = []

for i in range(len(w2v_all)):
    new_w2v_all.append(w2v_all[i][0] + w2v_all[i][1])
    
print('Length of new_w2v_all', len(new_w2v_all))
print('Length of inner list', len(new_w2v_all[0]))

Length of new_w2v_all 338
Length of inner list 898


In [27]:
fitles['w2v_all'] = new_w2v_all
fitles.sample()

,title,good_title,all_pos,dp1_pos,w2v,w2v_dp1,w2v_all
268,tumbling in the hay,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.0227402654, 0.00145335274, 0.00721915346, ...","[-0.0227402654, 0.00145335274, 0.00721915346, ...","[-0.0227402654, 0.00145335274, 0.00721915346, ..."


In [29]:
datapath = '../data'

datapath_fitles = os.path.join(datapath, 'fourgram_titles_dataset.pkl')
fitles.to_pickle(datapath_fitles)